In [1]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
ws = Workspace.from_config()

In [2]:
ws

Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1')

In [3]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [4]:
# enter details of your AzureML workspace
subscription_id = ws.subscription_id
resource_group = ws.resource_group
workspace = ws.name

In [5]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:
# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
  name=local_endpoint_name, description="this is a sample local endpoint"
)

In [25]:
model = ws.models['titanic_model'] #Model(path="../model-1/model/sklearn_regression_model.pkl")
env = Environment(
    conda_file="scoringConfig.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

local_endpoint_name = 'titanic_test'
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
         scoring_script="score.py" #code="../model-1/onlinescoring",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [26]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (local-12191113726521) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'local-12191113726521', 'description': 'this is a sample local endpoint', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/local-12191113726521'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f4b802429d0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [27]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (titanic_test / blue) Done (0m 0s)


RequiredLocalArtifactsNotFoundError: ("Local endpoints only support local artifacts. '%s' did not contain required local artifact '%s' of type '%s'.", 'Local deployment (titanic_test / blue)', 'code_configuration.code', "<class 'str'>")

In [28]:
os.getenv("AZUREML_MODEL_DIR")

In [14]:
os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "model/titanic_model.pkl"
    )

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [15]:
ws.models

{'titanic_model': Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:9, version=9, tags={'data': 'titanic', 'model': 'classification'}, properties={})}

In [17]:
ws.

<bound method Workspace.get_details of Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1')>

In [19]:
from azureml.core import Model
Model.list(ws)

[Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:9, version=9, tags={'data': 'titanic', 'model': 'classification'}, properties={}),
 Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:8, version=8, tags={'data': 'titanic', 'model': 'classification'}, properties={}),
 Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:7, version=7, tags={'data': 'titanic', 'model': 'classification'}, properties={}),
 Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:6, version=6, tags=

In [20]:
Model.get_model_path(
        model_name="titanic_model", version=9)

Model titanic_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp1812/code. For more info, set logging level to DEBUG.



WebserviceException: WebserviceException:
	Message: Model titanic_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp1812/code. For more info, set logging level to DEBUG.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Model titanic_model not found in cache at azureml-models or in current working directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp1812/code. For more info, set logging level to DEBUG."
    }
}

In [22]:
model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'titanic_model.pkl')

TypeError: expected str, bytes or os.PathLike object, not NoneType